In [1]:
from sage.manifolds.utilities import ExpressionNice as EN

In [2]:
def formal_diff(f, x):
    if f!=0:
        tempX = SR.temp_var()
        return f.subs({x: tempX}).diff(tempX).subs({tempX: x})
    if f==0:
        return 0

In [3]:
#Definimos variables y funciones
var('eps', latex_name = '\\varepsilon')
var('t')
var('r')
var('th', latex_name = '\\theta')
var('ph', latex_name = '\\varphi')
coords = (t,r,th,ph)

h = function('h')(r,th)
k = function('k')(r,th)
v = function('v')(r,th)
omega = function('omega', latex_name='\\omega')(r)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

#Definimos las funciones N, V, K y Nphi (ecuación 23 AC)
Ne = e^(nu/2)*(1+1/2*eps^2*h)
Ve = e^(lamb)*(1+eps^2*v)
Ke = r^2*(1+eps^2*k)
Nphie = eps*omega

# <center> Tensor de Einstein </center>

In [4]:
gdd = matrix(SR, 4, 4)

gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

guu = 1/gdd

K1 = matrix(SR, 4, 4)
K1[0,3] = -omega*r^2*sin(th)^2
K1[3,0] = -omega*r^2*sin(th)^2

K2 = matrix(SR, 4, 4)
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*v
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

In [5]:
gedd = matrix(SR, 4, 4)
for ii in range(0,4):
    for jj in range(0,4):
        gedd[ii,jj] = gdd[ii,jj] + eps*K1[ii,jj] + 1/2*eps^2*K2[ii,jj]
        
geuu = 1/gedd
    
chris = [[[0 for kk in range(0,4)] for jj in range(0,4)] for ii in range(0,4)]
for ii in range(0,4):
    for jj in range(0,4):
        for kk in range(0,4):
            chris[ii][jj][kk] = 0.5*sum(geuu[ii,dd]*(formal_diff(gedd[dd,jj],coords[kk])
                                                   + formal_diff(gedd[dd,kk],coords[jj])
                                                   - formal_diff(gedd[jj,kk],coords[dd])) for dd in range(0,4))

riem = [[[[0 for ll in range(0,4)] for kk in range(0,4)] for jj in range(0,4)] for ii in range(0,4)]
for ii in range(0,4):
    for jj in range(0,4):
        for kk in range(0,4):
            for ll in range(0,4):
                riem[ii][jj][kk][ll] = (formal_diff(chris[ii][jj][ll],coords[kk])
                                      - formal_diff(chris[ii][jj][kk],coords[ll])
                                      + sum(chris[ii][kk][dd]*chris[dd][ll][jj]
                                      - chris[ii][ll][dd]*chris[dd][kk][jj] for dd in range(0,4)))

ricci = matrix(SR, 4, 4)
for ii in range(0,4):
    for kk in range(0,4):
        ricci[ii,kk] = sum(riem[dd][ii][dd][kk] for dd in range(0,4))

s_curv = sum(sum(ricci[ii,jj]*geuu[ii,jj] for ii in range(0,4)) for jj in range(0,4))

Ge = matrix(SR, 4, 4)
Ge = ricci - 0.5*gedd*s_curv

dGe = matrix(SR, 4, 4)
ddGe = matrix(SR, 4, 4)
for ii in range(0,4):
    for jj in range(0,4):
        dGe[ii,jj] = formal_diff(Ge[ii,jj],eps)
        ddGe[ii,jj] = formal_diff(formal_diff(Ge[ii,jj],eps),eps)

# <center> Tensor de energía-momento </center>

In [6]:
#Funciones dependientes de eps
Omegane = function('Omegane', latex_name='\\Omega_{n\\varepsilon}')(eps)
Omegape = function('Omegape', latex_name='\\Omega_{p\\varepsilon}')(eps)

Nne = function('Nne', latex_name='N_{n\\varepsilon}')(r,th,eps)
Npe = function('Npe', latex_name='N_{p\\varepsilon}')(r,th,eps)

#Tomando eps=0
Nn = function('Nn', latex_name='N_{n}')(r,th)
Np = function('Np', latex_name='N_{p}')(r,th)

sus_func = [eps==0,
            Omegane.subs(eps==0)==0, #Staticity at the background
            Nne.subs(eps==0)==Nn,
            Omegape.subs(eps==0)==0, #Staticity at the background
            Npe.subs(eps==0)==Np]

## <center> Campo $u$ de los neutrones </center>

In [7]:
uevec = [Nne,0,0,Nne*Omegane]
Uevec = [0,0,0,0]

for ii in range(0,4):
    for jj in range(0,4):
        Uevec[ii] += uevec[jj]*gedd[ii,jj]

norm = 0
for ii in range(0,4):
    norm += uevec[ii]*Uevec[ii]

Nne_value = sqrt(solve(norm==-1, Nne^2)[0].rhs())

for ii in range(4):
    uevec[ii] = uevec[ii].subs(Nne==Nne_value)
    Uevec[ii] = Uevec[ii].subs(Nne==Nne_value)

Nn_value = copy(Nne_value)
for ii in range(len(sus_func)):
    Nn_value = Nn_value.subs(sus_func[ii])

sus_func.extend([Nn==Nn_value])

uvec = copy(uevec)
for ii in range(4):
    for jj in range(len(sus_func)):
        uvec[ii] = uvec[ii].subs(sus_func[jj])

Nne_paper = 1/sqrt(Ne^2 - sin(th)^2*Ke*(Nphie-Omegane)^2)

uvec_paper = [Nne_paper,0,0,Omegane*Nne_paper]
for ii in range(4):
    for jj in range(len(sus_func)):
        uvec_paper[ii] = uvec_paper[ii].subs(sus_func[jj])

uvec_check = [x-y for (x,y) in zip(uvec, uvec_paper)]

show(LatexExpr("u \\rightarrow "), bool(uvec_check==[0,0,0,0]))

u \rightarrow  True

In [8]:
var('Omegan1', latex_name="\\Omega^{(1)}_{n}")

sus_func.extend([formal_diff(formal_diff(Omegane,eps),eps).subs(eps==0) == 0, #En nuestro gauge Omega2=0
                 formal_diff(Omegane,eps).subs(eps==0) == Omegan1])

uvec1 = [0,0,0,0]
uvec2 = [0,0,0,0]

for ii in range(4):
    uvec1[ii] = formal_diff(uevec[ii],eps)
    uvec2[ii] = formal_diff(formal_diff(uevec[ii],eps),eps)

for ii in range(4):
    for jj in range(len(sus_func)):
        uvec1[ii] = uvec1[ii].subs(sus_func[jj])
        uvec2[ii] = uvec2[ii].subs(sus_func[jj])

uvec1_paper = [0,0,0,0]
uvec1_paper[0] = 1/2*e^(-3*nu/2)*K1[0,0]
uvec1_paper[3] = e^(-nu/2)*Omegan1

uvec2_paper = [0,0,0,0]
uvec2_paper[0] = e^(-3*nu/2)*(1/2*K2[0,0]+3/4*e^(-nu)*K1[0,0]+2*Omegan1*K1[0,3]+Omegan1^2*r^2*sin(th)^2)

uvec1_check = [ii-jj for (ii,jj) in zip(uvec1, uvec1_paper)]
uvec2_check = [ii-jj for (ii,jj) in zip(uvec2, uvec2_paper)]

show(LatexExpr("u^{(1)} \\rightarrow"), uvec1_check==[0,0,0,0])
show(LatexExpr("u^{(2)} \\rightarrow"), uvec2_check==[0,0,0,0])

u^{(1)} \rightarrow True

u^{(2)} \rightarrow True

## <center> Campo $v$ de los protones </center>

In [9]:
vevec = [Npe,0,0,Npe*Omegape]
Vevec = [0,0,0,0]

for ii in range(4):
    for jj in range(4):
        Vevec[ii] += vevec[jj]*gedd[ii,jj]

norm = 0
for ii in range(4):
    norm += vevec[ii]*Vevec[ii]

Npe_value = sqrt(solve(norm==-1, Npe^2)[0].rhs())
for ii in range(4):
    vevec[ii] = vevec[ii].subs(Npe==Npe_value)
    Vevec[ii] = Vevec[ii].subs(Npe==Npe_value)

Np_value = copy(Npe_value)
for ii in range(len(sus_func)):
    Np_value = Np_value.subs(sus_func[ii])

sus_func.extend([Np==Np_value])

vvec = copy(vevec)
for ii in range(4):
    for jj in range(len(sus_func)):
        vvec[ii] = vvec[ii].subs(sus_func[jj])

Npe_paper = 1/sqrt(Ne^2 - sin(th)^2*Ke*(Nphie-Omegape)^2)

vvec_paper = [Npe_paper,0,0,Omegape*Npe_paper]

for ii in range(4):
    for jj in range(len(sus_func)):
        vvec_paper[ii] = vvec_paper[ii].subs(sus_func[jj])

vvec_check = [x-y for (x,y) in zip(vvec, vvec_paper)]

show(LatexExpr("v \\rightarrow "), bool(vvec_check==[0,0,0,0]))

v \rightarrow  True

In [10]:
var('Omegap1', latex_name="\\Omega^{(1)}_{p}")

sus_func.extend([formal_diff(formal_diff(Omegape,eps),eps).subs(eps==0) == 0, #En nuestro gauge Omega2=0
                 formal_diff(Omegape,eps).subs(eps==0) == Omegap1])

vvec1 = [0,0,0,0]
vvec2 = [0,0,0,0]

for ii in range(4):
    vvec1[ii] = formal_diff(vevec[ii],eps)
    vvec2[ii] = formal_diff(formal_diff(vevec[ii],eps),eps)

for ii in range(4):
    for jj in range(len(sus_func)):
        vvec1[ii] = vvec1[ii].subs(sus_func[jj])
        vvec2[ii] = vvec2[ii].subs(sus_func[jj])

vvec1_paper = [0,0,0,0]
vvec1_paper[0] = 1/2*e^(-3*nu/2)*K1[0,0]
vvec1_paper[3] = e^(-nu/2)*Omegap1

vvec2_paper = [0,0,0,0]
vvec2_paper[0] = e^(-3*nu/2)*(1/2*K2[0,0]+3/4*e^(-nu)*K1[0,0]+2*Omegap1*K1[0,3]+Omegap1^2*r^2*sin(th)^2)
vvec2_paper[3] = e^(-nu/2)*(e^(-nu)*K1[0,0]*Omegap1)  

vvec1_check = [ii-jj for (ii,jj) in zip(vvec1, vvec1_paper)]
vvec2_check = [ii-jj for (ii,jj) in zip(vvec2, vvec2_paper)]

show(LatexExpr("v^{(1)} \\rightarrow"), vvec1_check==[0,0,0,0])
show(LatexExpr("v^{(2)} \\rightarrow"), vvec2_check==[0,0,0,0])

v^{(1)} \rightarrow True

v^{(2)} \rightarrow True

## <center> Ecuación de Einstein </center>

In [11]:
n = function('n')(r,th,eps)
p = function('p')(r,th,eps)

n0 = function('n0', latex_name="n_0")(r)
p0 = function('p0', latex_name="p_0")(r)

eta = function('eta', latex_name="\\eta")(r,th)
Phi = function('Phi', latex_name="\\Phi")(r,th)

n = n0 + 1/2*eps^2*eta
p = p0 + 1/2*eps^2*Phi
x = sqrt(n*p)

Lambdae = function('Lambdae', latex_name='\\Lambda_{\\varepsilon}')(n^2, p^2, x^2)
Psie = function('Psie', latex_name='\\Psi_{\\varepsilon}')(n^2, p^2, x^2)

nevec = [n*ii for ii in uevec] #eq 13 AC
pevec = [p*ii for ii in vevec] #eq 13 AC

ned = [0,0,0,0]
ped = [0,0,0,0]
for ii in range(4):
    for jj in range(4):
        ned[ii] += nevec[jj]*gedd[ii,jj]
        ped[ii] += pevec[jj]*gedd[ii,jj]

nd = copy(ned)
pd = copy(ped)
for ii in range(4):
    for jj in range(len(sus_func)):
        nd[ii] = nd[ii].subs(sus_func[jj])
        pd[ii] = pd[ii].subs(sus_func[jj])

A = -formal_diff(Lambdae, x^2)
B = -2*formal_diff(Lambdae, n^2)
C = -2*formal_diff(Lambdae, p^2)

mued = [B*ii for ii in ned] + [A*jj for jj in ped] #eq 2 C
chied = [C*ii for ii in ped] + [A*jj for jj in ned] #eq 2 C

muevec = [0,0,0,0]
chievec = [0,0,0,0]
for ii in range(4):
    for jj in range(4):
        muevec[ii] += mued[jj]*gedd[ii,jj]
        chievec[ii] += chied[jj]*gedd[ii,jj]

Psie = Lambdae - sum([ii*jj for (ii,jj) in zip(nevec, mued)]) - sum([ii*jj for (ii,jj) in zip(pevec, chied)])

LambdaePsie = Psie - Lambdae

In [12]:
dirac = diagonal_matrix([1, 1, 1, 1])

Teud = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        Teud[ii,jj] = Psie*dirac[ii,jj] + pevec[ii]*chied[jj] + nevec[ii]*mued[jj]

Te, dTe, ddTe = matrix(SR, 4, 4), matrix(SR, 4, 4), matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        for kk in range(4):
            Te[ii,jj] += Teud[kk,jj]*gedd[kk,ii]
        
        dTe[ii,jj] = formal_diff(Te[ii,jj],eps)
        ddTe[ii,jj] = formal_diff(formal_diff(Te[ii,jj],eps),eps)

kappa = 8*pi

In [13]:
#dLambdae_dr = cadena(Lambdae, r)
#ddLambdae_dn2_dr = formal_diff(cadena(Lambdae, r), n2)
#ddLambdae_dp2_dr = formal_diff(cadena(Lambdae, r), p2)
#ddLambdae_dx2_dr = formal_diff(cadena(Lambdae, r), x2)
#for ii in range(len(sus_func)):
#    dLambdae_dr = dLambdae_dr.subs(sus_func[ii])
#    ddLambdae_dn2_dr = ddLambdae_dn2_dr.subs(sus_func[ii])
#    ddLambdae_dp2_dr = ddLambdae_dp2_dr.subs(sus_func[ii])
#    ddLambdae_dx2_dr = ddLambdae_dx2_dr.subs(sus_func[ii])
#
#sus_func.extend([formal_diff(Lambdae, r).subs(eps==0) == dLambdae_dr,
#                 formal_diff(Lambdae, n2, r).subs(eps==0) == ddLambdae_dn2_dr,
#                 formal_diff(Lambdae, p2, r).subs(eps==0) == ddLambdae_dp2_dr,
#                 formal_diff(Lambdae, x2, r).subs(eps==0) == ddLambdae_dx2_dr])

In [14]:
mue = B*n + A*p #eq 22 C
chie = A*n + C*p #eq 22 C

#Volvemos a definir mu_d y chi_d porque los que habíamos definido antes eran de AC y este es de C
mued = [mue*ii for ii in Uevec] #eq 17 C
chied = [chie*ii for ii in Uevec] #eq 17 C

muinf = -mued[0] #eq 20
chiinf = -chied[0] #eq 20
for ii in range(len(sus_func)):
    muinf = muinf.subs(sus_func[ii])
    chiinf = chiinf.subs(sus_func[ii])

eq_mu = formal_diff(muinf,r)
eq_chi = formal_diff(chiinf,r)
for ii in range(len(sus_func)):
    eq_mu = eq_mu.subs(sus_func[ii])
    eq_chi = eq_chi.subs(sus_func[ii])

A0, B0, C0 = A, B, C

A00 = A + 2*formal_diff(B,p^2)*n*p + 2*formal_diff(A,n^2)*n^2 + 2*formal_diff(A,p^2)*p^2 + formal_diff(A,x^2)*p*n
B00 = B + 2*formal_diff(B,n^2)*n^2 + 4*formal_diff(A,n^2)*n*p + formal_diff(A,x^2)*p^2
C00 = C + 2*formal_diff(C,p^2)*p^2 + 4*formal_diff(A,p^2)*n*p + formal_diff(A,x^2)*n^2

A000, B000, C000 = A00, B00, C00
for ii in range(len(sus_func)):
    A0 = A0.subs(sus_func[ii])
    B0 = B0.subs(sus_func[ii])
    C0 = C0.subs(sus_func[ii])
    A000 = A000.subs(sus_func[ii])
    B000 = B000.subs(sus_func[ii])
    C000 = C000.subs(sus_func[ii])

eq26_1 = A000*formal_diff(p0,r) + B000*formal_diff(n0,r) + 1/2*(B0*n0 + A0*p0)*formal_diff(nu,r)
eq26_2 = C000*formal_diff(p0,r) + A000*formal_diff(n0,r) + 1/2*(A0*n0 + C0*p0)*formal_diff(nu,r)
for ii in range(len(sus_func)):
    eq26_1 = eq26_1.subs(sus_func[ii])
    eq26_2 = eq26_2.subs(sus_func[ii])

eq_mu = eq_mu*e^(-nu/2)
eq_mu_check = eq_mu - eq26_1
eq_chi = eq_chi*e^(-nu/2)
eq_chi_check = eq_chi - eq26_2

show(LatexExpr("\\text{Eq 26.1} \\rightarrow"), bool(eq_mu_check==0))
show(LatexExpr("\\text{Eq 26.2} \\rightarrow"), bool(eq_chi_check==0))

\text{Eq 26.1} \rightarrow True

\text{Eq 26.2} \rightarrow True

In [15]:
dp0_value = solve(eq26_1, formal_diff(p0,r))[0].rhs()
for ii in range(len(sus_func)):
    dp0_value = dp0_value.subs(sus_func[ii])
sus_func.extend([formal_diff(p,r).subs(eps==0)==dp0_value])

dn0_value = solve(eq26_2, diff(n0,r))[0].rhs()
for ii in range(len(sus_func)):
    dn0_value = dn0_value.subs(sus_func[ii])
sus_func.extend([formal_diff(n,r).subs(eps==0)==dn0_value])

# <center> $G_{\mu\nu} = \kappa T_{\mu\nu}$ </center>

In [16]:
Lambda0 = function('Lambda0', latex_name='\\Lambda_0')(n^2,p^2,x^2)
Psi0 = function('Psi0', latex_name="\\Psi_0")(n^2,p^2,x^2)

sus_func.extend([Lambdae.subs(eps==0) == Lambda0])

Psi0 = copy(Psie)
for ii in range(len(sus_func)):
    Psi0 = Psi0.subs(sus_func[ii])

G = copy(Ge)
T = copy(Te)
for ii in range(len(sus_func)):
    G = G.subs(sus_func[ii])
    T = T.subs(sus_func[ii])

## <center> $G_{00} = \kappa T_{00}$ </center>

In [17]:
dlamb_value = solve(G[0,0] - kappa*T[0,0] == 0, formal_diff(lamb, r))[0].rhs().simplify_full()
dlamb_paper = (1-e^(lamb))/r - 8*pi*r*e^(lamb)*Lambda0
dlamb_check = dlamb_value - dlamb_paper
show(LatexExpr("\\lambda' \\rightarrow"), bool(dlamb_check==0))

for ii in range(len(sus_func)):
    dlamb_value = dlamb_value.subs(sus_func[ii])

sus_func.extend([formal_diff(lamb,r)==dlamb_value])

\lambda' \rightarrow True

## <center> $G_{11} = \kappa T_{11}$ </center>


In [18]:
dnu_value_11 = solve(G[1,1] - kappa*T[1,1] == 0, formal_diff(nu, r))[0].rhs().simplify_full()
dnu_paper = -(1-e^(lamb))/r + 8*pi*r*e^(lamb)*Psi0
dnu_check_11 = dnu_value_11 - dnu_paper
show(LatexExpr("\\nu' \\rightarrow"), bool(dnu_check_11==0))

for ii in range(len(sus_func)):
    dnu_value_11 = dnu_value_11.subs(sus_func[ii])

sus_func.extend([formal_diff(nu,r)==dnu_value_11])

\nu' \rightarrow True

In [19]:
#dnu_value_11 = solve(G[1,1] - kappa*T[1,1] == 0, diff(nu, r))[0].rhs().simplify_full()
#ddnu_value_11 = diff(dnu_value_11,r).simplify_full()
#show(ddnu_value_11)
#for ii in range(len(sus_func)):
#    ddnu_value_11 = ddnu_value_11.subs(sus_func[ii])
#print(ddnu_value_11.simplify_full())
#dnu_paper = -(1-e^(lamb))/r + 8*pi*r*e^(lamb)*Psi0
#dnu_check_11 = dnu_value_11 - dnu_paper
#show(LatexExpr("\\nu' \\rightarrow"), bool(dnu_check_11==0))
#
#for ii in range(len(sus_func)):
#    dnu_value_11 = dnu_value_11.subs(sus_func[ii])

## <center> $G_{11} = \kappa T_{11}$ y $G_{22} = \kappa T_{22}$ </center>


In [20]:
dnu_value_22 = solve(G[2,2] - kappa*T[2,2] == 0, formal_diff(nu, r))[0].rhs().simplify_full()

#Despejamos la raiz cuadrada de la ecuación
dEq_1122 = -(dnu_value_11 - dnu_value_22 == 0)*e^(nu)*2*r
dEq_1122 = dEq_1122 + sqrt(-256*pi*r^2*e^(lamb + 2*nu)*n0^2*formal_diff(Lambdae,n^2).subs(eps==0)\
                           -256*pi*r^2*e^(lamb + 2*nu)*p0^2*formal_diff(Lambdae,p^2).subs(eps==0)\
                           +r^2*e^(2*nu)*formal_diff(lamb, r)^2\
                           -8*r^2*e^(2*nu)*formal_diff(formal_diff(nu, r), r)\
                           +4*r*e^(2*nu)*formal_diff(lamb, r)\
                           +4*(32*pi*r^2*Lambda0*e^lamb + 1)*e^(2*nu))

for ii in range(len(sus_func)):
    dEq_1122 = dEq_1122.subs(sus_func[ii])

ddnu_value = solve(dEq_1122^2, formal_diff(formal_diff(nu,r),r))[0].rhs().simplify_full()

ddnu_paper = 2*r*formal_diff(formal_diff(nu,r),r)\
           + formal_diff(nu,r)*(r*formal_diff(nu,r)-2)\
           - formal_diff(lamb,r)*(2+r*formal_diff(nu,r))\
           + 4/r*(e^(lamb)-1) == 0
ddnu_paper = solve(ddnu_paper, formal_diff(formal_diff(nu,r),r))[0].rhs()
for ii in range(len(sus_func)):
    ddnu_paper = ddnu_paper.subs(sus_func[ii])
    
ddnu_check = ddnu_value - ddnu_paper
show(LatexExpr("\\nu'' \\rightarrow"), bool(ddnu_check==0))

sus_func.extend([formal_diff(formal_diff(nu,r),r)==ddnu_value])

\nu'' \rightarrow True

# <center> $G^{\prime}_{\mu\nu} = \kappa T^{\prime}_{\mu\nu}$ </center>

In [21]:
dLambdaePsie = formal_diff(LambdaePsie, eps)
for ii in range(len(sus_func)):
    dLambdaePsie = dLambdaePsie.subs(sus_func[ii])

In [22]:
j = e^(-(lamb+nu)/2)
M = r*(1-e^(-lamb))/2

dG = copy(dGe)
dT = copy(dTe)
for ii in range(0,len(sus_func)):
    dG = dG.subs(sus_func[ii])
    dT = dT.subs(sus_func[ii])

In [23]:
Lambda1_value = formal_diff(Lambdae, eps)
for ii in range(len(sus_func)):
    Lambda1_value = Lambda1_value.subs(sus_func[ii])

Lambda1_paper = 0
Lambda1_check = Lambda1_value - Lambda1_paper

show(LatexExpr("\\Lambda^{(1)} = "), Lambda1_value, LatexExpr("\\rightarrow"), bool(Lambda1_check==0))

\Lambda^{(1)} =  0 \rightarrow True

In [24]:
Psi1_value = formal_diff(Lambdae, eps) + formal_diff(LambdaePsie, eps)
for ii in range(len(sus_func)):
    Psi1_value = Psi1_value.subs(sus_func[ii])

Psi1_paper = 0
Psi1_check = Psi1_value - Psi1_paper

show(LatexExpr("\\Psi^{(1)} = "), Psi1_value, LatexExpr("\\rightarrow"), bool(Psi1_check==0))

\Psi^{(1)} =  0 \rightarrow True

In [27]:
#De dEq el único valor que no es idénticamente nulo es 03 (y 30)
dEq_03 = dG[0,3] - kappa*dT[0,3] == 0
for ii in range(len(sus_func)):
    dEq_03 = dEq_03.subs(sus_func[ii])
dEq_03 = dEq_03.simplify_full()

ddomega_value = solve(dEq_03, formal_diff(formal_diff(omega,r),r))[0].rhs().simplify_full()

Ln = (omega - Omegan1)*eps
chi0 = C*p + A*n

ddomega_paper = 1/r^4*formal_diff(r^4*e^(-(lamb+nu)/2)*formal_diff(Ln,r),r)\
              - 16*pi*e^((lamb-nu)/2)*(Psi0-Lambda0)*Ln\
              - 16*pi*e^((lamb-nu)/2)*chi0*p0*(Omegan1-Omegap1)*eps == 0
ddomega_paper = solve(ddomega_paper, formal_diff(formal_diff(omega,r),r))[0].rhs().simplify_full()

for ii in range(len(sus_func)):
    ddomega_value = ddomega_value.subs(sus_func[ii])
    ddomega_paper = ddomega_paper.subs(sus_func[ii])

ddomega_check = ddomega_value - ddomega_paper
for ii in range(len(sus_func)):
    ddomega_value = ddomega_value.subs(sus_func[ii])
    ddomega_paper = ddomega_paper.subs(sus_func[ii])
    ddomega_check = ddomega_check.subs(sus_func[ii])

show(ddomega_value.simplify_full())
show(ddomega_paper.simplify_full())
show(bool(ddomega_check==0))

show(LatexExpr("\\omega'' = "), EN(ddomega_check.simplify_full()))

#sus_func.extend([diff(omega,r,r)==ddomega_value])

4*(8*pi*Omegan1*r*e^lamb(r)*n0(r)^2*D[0](Lambdae)(n0(r)^2, p0(r)^2, n0(r)*p0(r)) - 8*pi*r*e^lamb(r)*n0(r)^2*omega(r)*D[0](Lambdae)(n0(r)^2, p0(r)^2, n0(r)*p0(r)) - 2*(pi*r^2*e^lamb(r)*p0(r)^2*diff(omega(r), r) - 4*(pi*Omegap1*r*e^lamb(r) - pi*r*e^lamb(r)*omega(r))*p0(r)^2)*D[1](Lambdae)(n0(r)^2, p0(r)^2, n0(r)*p0(r)) - (2*pi*r^2*e^lamb(r)*n0(r)^2*D[0](Lambdae)(n0(r)^2, p0(r)^2, n0(r)*p0(r)) + 1)*diff(omega(r), r))/r

4*(8*pi*Omegan1*r*e^lamb(r)*n0(r)^2*D[0](Lambdae)(n0(r)^2, p0(r)^2, n0(r)*p0(r)) - 8*pi*r*e^lamb(r)*n0(r)^2*omega(r)*D[0](Lambdae)(n0(r)^2, p0(r)^2, n0(r)*p0(r)) - 4*(pi*Omegan1 - pi*Omegap1)*r*e^lamb(r)*n0(r)*p0(r)*D[2](Lambdae)(n0(r)^2, p0(r)^2, n0(r)*p0(r)) - 2*(pi*r^2*e^lamb(r)*p0(r)^2*diff(omega(r), r) - 4*(pi*Omegap1*r*e^lamb(r) - pi*r*e^lamb(r)*omega(r))*p0(r)^2)*D[1](Lambdae)(n0(r)^2, p0(r)^2, n0(r)*p0(r)) - (2*pi*r^2*e^lamb(r)*n0(r)^2*D[0](Lambdae)(n0(r)^2, p0(r)^2, n0(r)*p0(r)) + 1)*diff(omega(r), r))/r

False

\omega'' =  16*(pi*Omegan1 - pi*Omegap1)*e^lamb(r)*n0(r)*p0(r)*d(Lambdae)/d(n0(r)*p0(r))

In [ ]:
sys.exit()

# <center> $G^{\prime\prime}_{\mu\nu} = \kappa T^{\prime\prime}_{\mu\nu}$ </center>

In [ ]:
ddLambdaePsie = diff(LambdaePsie, eps, eps)
for ii in range(len(sus_func)):
    ddLambdaePsie = ddLambdaePsie.subs(sus_func[ii])

In [ ]:
Lambda2 = function('Lambda2', latex_name='\\Lambda^{(2)}')(n2,p2,x2)

sus_func.extend([diff(Lambdae,eps,eps).subs(eps==0) == Lambda2])

ddG = copy(ddGe)
ddT = copy(ddTe)
for ii in range(len(sus_func)):
    ddG = ddG.subs(sus_func[ii])
    ddT = ddT.subs(sus_func[ii])

In [ ]:
sys.exit()

## <center> Descomposición en $P_l(\cos(\theta))$ </center>

In [ ]:
# FALTA ECUACIÓN 31 DE ETA Y PHI

In [ ]:
h0 = function('h0', latex_name='h_0')(r)
v0 = function('v0', latex_name='v_0')(r)

h2 = function('h2', latex_name='h_2')(r)
k2 = function('k2', latex_name='k_2')(r)
v2 = function('v2', latex_name='v_2')(r)

Lambda20 = function('Lambda20', latex_name='\\Lambda_{0}^{(2)}')(n2,p2,x2)
Lambda22 = function('Lambda22', latex_name='\\Lambda_{2}^{(2)}')(n2,p2,x2)

LP1 = function('LP1', latex_name='LP_1')(th)
LP2 = function('LP2', latex_name='LP_2')(th)

lp2 = (3*cos(th)^2-1)/2

In [ ]:
sus_descomp = [v==v0+v2*lp2,
               diff(v,r)==diff(v0+v2*lp2,r),
               diff(v,r,r)==diff(v0+v2*lp2,r,r),
               diff(v,th)==diff(v0+v2*lp2,th),
               diff(v,th,th)==diff(v0+v2*lp2,th,th),
               diff(v,th,r)==diff(v0+v2*lp2,th,r),
               diff(v,r,th)==diff(v0+v2*lp2,r,th),
               k==k2*lp2,
               diff(k,r)==diff(k2*lp2,r),
               diff(k,r,r)==diff(k2*lp2,r,r),
               diff(k,th)==diff(k2*lp2,th),
               diff(k,th,th)==diff(k2*lp2,th,th),
               diff(k,th,r)==diff(k2*lp2,th,r),
               diff(k,r,th)==diff(k2*lp2,r,th),
               h==h0+h2*lp2,
               diff(h,r)==diff(h0+h2*lp2,r),
               diff(h,r,r)==diff(h0+h2*lp2,r,r),
               diff(h,th)==diff(h0+h2*lp2,th),
               diff(h,th,th)==diff(h0+h2*lp2,th,th),
               diff(h,th,r)==diff(h0+h2*lp2,th,r),
               diff(h,r,th)==diff(h0+h2*lp2,r,th),
               Lambda2==Lambda20+Lambda22*lp2,
               diff(Lambda2,r)==diff(Lambda20+Lambda22*lp2,r),
               diff(Lambda2,r,r)==diff(Lambda20+Lambda22*lp2,r,r),
               diff(Lambda2,th)==diff(Lambda20+Lambda22*lp2,th),
               diff(Lambda2,th,th)==diff(Lambda20+Lambda22*lp2,th,th),
               diff(Lambda2,th,r)==diff(Lambda20+Lambda22*lp2,th,r),
               diff(Lambda2,r,th)==diff(Lambda20+Lambda22*lp2,r,th),
               Psi2==Psi20+Psi22*lp2,
               diff(Psi2,r)==diff(Psi20+Psi22*lp2,r),
               diff(Psi2,r,r)==diff(Psi20+Psi22*lp2,r,r),
               diff(Psi2,th)==diff(Psi20+Psi22*lp2,th),
               diff(Psi2,th,th)==diff(Psi20+Psi22*lp2,th,th),
               diff(Psi2,th,r)==diff(Psi20+Psi22*lp2,th,r),
               diff(Psi2,r,th)==diff(Psi20+Psi22*lp2,r,th)]

for ii in range(0,len(sus_descomp)):
    ddG = ddG.subs(sus_descomp[ii])
    ddT = ddT.subs(sus_descomp[ii])

ddG = ddG.simplify_full()
ddT = ddT.simplify_full()